<a href="https://colab.research.google.com/github/dsrestrepo/Enfasis2/blob/master/CNN_Enfa2(dogs%2Ccats%2CgorillaButHardToProcessing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
data_entrenamiento = '/content/drive/My Drive/ML_enfasis_V2/data/entrenamiento'
data_validacion = '/content/drive/My Drive/ML_enfasis_V2/data/validacion'

"""
Parameters
"""
epocas=20
longitud, altura = 100, 100
batch_size = 32
pasos = 700
validation_steps = 300
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 3
lr = 0.0004


In [0]:
##Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 999 images belonging to 3 classes.
Found 2043 images belonging to 3 classes.


In [0]:
cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 64)        0         
Total params: 9,152
Trainable params: 9,152
Non-trainable params: 0
_________________________________________________________________


In [0]:
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))
cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 40000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               10240256  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)              

In [0]:
cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

Epoch 1/20
700/700 [==============================] - 466s 666ms/step - loss: 0.6035 - acc: 0.6949 - val_loss: 0.7146 - val_acc: 0.6880
Epoch 2/20
700/700 [==============================] - 464s 663ms/step - loss: 0.2957 - acc: 0.8786 - val_loss: 0.7906 - val_acc: 0.7082
Epoch 3/20
700/700 [==============================] - 462s 659ms/step - loss: 0.1362 - acc: 0.9519 - val_loss: 0.9771 - val_acc: 0.7112
Epoch 4/20
700/700 [==============================] - 462s 661ms/step - loss: 0.0757 - acc: 0.9736 - val_loss: 1.1079 - val_acc: 0.7271
Epoch 5/20
700/700 [==============================] - 461s 659ms/step - loss: 0.0588 - acc: 0.9805 - val_loss: 1.2744 - val_acc: 0.7176
Epoch 6/20
700/700 [==============================] - 462s 660ms/step - loss: 0.0461 - acc: 0.9852 - val_loss: 1.2581 - val_acc: 0.7013
Epoch 7/20
700/700 [==============================] - 461s 659ms/step - loss: 0.0475 - acc: 0.9844 - val_loss: 1.3741 - val_acc: 0.7244
Epoch 8/20
700/700 [============================

In [0]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

In [0]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

longitud, altura = 150, 150
modelo = './modelo/modelo.h5'
pesos_modelo = './modelo/pesos.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)


Using TensorFlow backend.


OSError: ignored

In [0]:
def predict(file):
  x = load_img(file, target_size=(longitud, altura))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = cnn.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("pred: Perro")
  elif answer == 1:
    print("pred: Gato")
  elif answer == 2:
    print("pred: Gorila")

  return answer
predict('8.jpeg')

NameError: ignored